In [1]:
cd("/Users/jerobinett/Desktop/ERA5Analysis/Basin Data")
burrowactivate()
import ERA5Analysis as ERA
using PyCall, Dictionaries, DataFrames, Shapefile, JLD2, StaticArrays
@pyimport matplotlib.pyplot as plt
@pyimport matplotlib.patches as mpatches
@pyimport cartopy.crs as ccrs

  Activating project at `~/Desktop/ERA5Analysis`


In [3]:

#use for allowed ids

huc_paths = "HUC_Shapes/WBDHU".*string.([6,8]).*".shp"
shapes = Dictionary([6,8],DataFrame.(Shapefile.Table.(huc_paths)))

goodproj = ccrs.AlbersEqualArea(central_longitude = -147,
standard_parallels = (57,69))

gnomon = ccrs.Gnomonic(central_longitude = -147, central_latitude = 64)

ax = plt.subplot(1,1,1,projection = gnomon)

basincolors = Dictionary(ERA.basin_names,["red","green","purple","orange","blue", "pink", "yellow","black"])

basin_to_polys = jldopen("basin_to_polys.jld2")["basin_to_polys"]

for basin in ERA.basin_names
    polys = basin_to_polys[basin]
    for poly in polys
        poly = reduce(vcat, collect.(permutedims.(poly)))
        poly_plot = mpatches.Polygon(poly, closed=true, alpha=0.7, facecolor = basincolors[basin],
        transform = ccrs.PlateCarree())
        ax.add_patch(poly_plot)
    end
end
ax.set_extent(ERA.ak_bounds, crs=ccrs.PlateCarree())
ax.gridlines()
ax.coastlines()
plt.savefig("basin_vis.png")